In this notebook, I implement a simple gradient descent solver for logistic regression with optional sample weighting. I apply it to the Wisconsin breast cancer classification dataset.

I experimented with newton-raphson, but encountered severe numerical instability problems computing the Hessian. I may revisit this.

In [149]:
import numpy as np
import polars as pl
import warnings

from sklearn.metrics import classification_report

In [57]:
from sklearn.model_selection import train_test_split

DIAGNOSIS_TO_INT_MAPPING = {'B': 0, 'M': 1}

INT_TO_DIAGNOSIS_MAPPING = {v: k for k, v in DIAGNOSIS_TO_INT_MAPPING.items()}

def prepare_breast_cancer_wisconsin_dataset(path: str):
    # load data
    data = pl.read_csv(path)

    # process data into feature matrix (X) and target vector (y)
    X = data.select(pl.exclude(['id', 'diagnosis', ''])).to_numpy()
    y = data['diagnosis'].replace_strict(old=DIAGNOSIS_TO_INT_MAPPING, return_dtype=pl.Int8).to_numpy()

    # split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1738)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = prepare_breast_cancer_wisconsin_dataset(path='../data/breast_cancer_wisconsin_diagnosis.csv')

X_train.shape, y_train.shape

((455, 30), (455,))

In [198]:
SIGMOID_Z_IMPUTATION_THRESHOLD = -20

class LogisticRegression:
    """
    A logistic regression model
    """
    def __init__(self, tol: float = .001, max_iter: int = 1000, lr: float = .01, lr_decay: float = .9999, dtype: np.dtype = np.float32):
        """Initialize a logistic regression model

        Args:
            tol (float, optional): tolerance for stopping. GD is considered converged when all partials are below tol. Defaults to .001.
            max_iter (int, optional): maximum number of training interations. Defaults to 1000.
            lr (float, optional): learning rate, to be multiplied with the gradient vector before each GD step. Defaults to .01.
            lr_decay (float, optional): learning rate decay coefficient. At each iteration lr *= lr_decay. Defaults to .9999.
            dtype (np.dtype, optional): data type for all internal numpy arrays. Defaults to np.float32.
        """
        self.tol = tol
        self.max_iter = max_iter
        self.lr = lr
        self.dtype = dtype
        self.lr_decay = lr_decay

    def _add_ones_column(self, X: np.ndarray) -> np.ndarray:
        return np.concat([np.ones(shape=(X.shape[0], 1)), X], axis = 1)
    
    def _sigmoid(self, z: np.ndarray) -> np.ndarray:
        result = np.zeros_like(z, dtype=self.dtype)

        stable_mask = z > SIGMOID_Z_IMPUTATION_THRESHOLD

        # only compute sigmoid for z above threshold to prevent overflow. Otherwise just sub 0
        result[stable_mask] = 1 / (1 + np.exp(-z[stable_mask]))
        
        return result

    def _forward(self, X: np.ndarray) -> np.ndarray:
        return self._sigmoid(self._add_ones_column(X) @ self.w)

    def fit(self, X: np.ndarray, y: np.ndarray, weights: np.ndarray = None, verbose: bool = False) -> None:
        """Fit model

        Args:
            X (np.ndarray): training features
            y (np.ndarray): target vector
            weights (np.ndarray, optional): sample weights. Defaults to None.
            verbose (bool, optional): whether to print warnings. Defaults to False.

        Raises:
            ValueError: if sample weight array does not match shape of target
        """
        if weights is not None and weights.shape != y.shape:
            raise ValueError('sample weight array must have same shape as target vector')
        
        X_one_col = self._add_ones_column(X)
        
        self.num_params = X_one_col.shape[1]

        self.w = np.random.uniform(size=(self.num_params,))
        
        descent_term = np.full(shape=(self.num_params,), fill_value=np.inf) # the f(x) / f'(x) term in newton's method

        # so I don't decay the class attribute
        _lr = self.lr

        for i in range(self.max_iter):
            if np.all(np.abs(descent_term) < self.tol):
                self.n_iters_to_fit = i + 1
                break
            
            # compute gradient vector (d loss wrt weights)
            if weights is not None:
                grad_w = X_one_col.T @ ((self._sigmoid(X_one_col @ self.w) - y) * weights)
            else:
                grad_w = X_one_col.T @ (self._sigmoid(X_one_col @ self.w) - y)
            
            # gradient descent update
            self.w -= grad_w * _lr

            # learning rate decay
            _lr *= self.lr_decay

        else:
            if verbose:
                warnings.warn('Gradient descent method failed to converge. Consider increasing max_iter and/or tol.')

    def predict(self, X: np.ndarray, probability_threshold: float = 0.5) -> np.ndarray:
        """Make predictions

        Args:
            X (np.ndarray): training features
            probability_threshold (float, optional): threshold above which samples are assigned to the positive class. Defaults to 0.5.

        Returns:
            np.ndarray: an array of shape (number of samples, ) containing binary class labels (in {0, 1})
        """
        return np.astype(self._forward(X) > probability_threshold, np.int8)


In [249]:
# no weights
model = LogisticRegression(max_iter=10_000)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.96      0.87      0.91        79
           1       0.76      0.91      0.83        35

    accuracy                           0.89       114
   macro avg       0.86      0.89      0.87       114
weighted avg       0.90      0.89      0.89       114



In [251]:
# weight positive class 3x negative class
model = LogisticRegression(max_iter=10_000)
model.fit(X_train, y_train, weights=y_train + .5)
y_test_pred = model.predict(X_test)

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94        79
           1       0.93      0.80      0.86        35

    accuracy                           0.92       114
   macro avg       0.93      0.89      0.90       114
weighted avg       0.92      0.92      0.92       114

